## Serving

In [2]:
# First code block - Import necessary libraries and set up OpenAI client
import openai
import os
from typing import Dict, List
from datetime import datetime

# Set your OpenAI API key
os.environ['OPENAI_API_KEY'] = 
client = openai.OpenAI()

# Define the model ID
FINE_TUNED_MODEL = "ft:gpt-4o-mini-2024-07-18:personal::Af1GA1or"

In [5]:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [3]:
class PokerStrategyService:
    def __init__(self, 
                 model_id: str = "ft:gpt-4o-mini-2024-07-18:personal::Af1GA1or",
                 system_prompt: str = "You are an expert poker strategist."):
        """
        Initialize the Poker Strategy Service
        
        Args:
            model_id (str): The ID of the fine-tuned model
            system_prompt (str): The system prompt to use for all queries
        """
        self.model_id = model_id
        self.system_prompt = system_prompt
        self.client = openai.OpenAI()
        
    def get_strategy_advice(self, 
                          query: str, 
                          temperature: float = 0.7,
                          max_retries: int = 3) -> Dict:
        """
        Get poker strategy advice using the fine-tuned model
        
        Args:
            query (str): The poker situation or question
            temperature (float): Controls randomness (0.0 to 1.0)
            max_retries (int): Maximum number of retry attempts
            
        Returns:
            Dict: Response containing model's advice and metadata
        """
        retry_count = 0
        while retry_count < max_retries:
            try:
                start_time = datetime.now()
                response = self.client.chat.completions.create(
                    model=self.model_id,
                    messages=[
                        {"role": "system", "content": self.system_prompt},
                        {"role": "user", "content": query}
                    ],
                    temperature=temperature
                )
                
                elapsed_time = (datetime.now() - start_time).total_seconds()
                
                return {
                    "status": "success",
                    "response": response.choices[0].message.content,
                    "model": self.model_id,
                    "latency": elapsed_time,
                    "tokens": {
                        "prompt_tokens": response.usage.prompt_tokens,
                        "completion_tokens": response.usage.completion_tokens,
                        "total_tokens": response.usage.total_tokens
                    }
                }
                
            except Exception as e:
                retry_count += 1
                if retry_count == max_retries:
                    return {
                        "status": "error",
                        "error": str(e),
                        "model": self.model_id
                    }
                continue
    
    def batch_process(self, queries: List[str], 
                     temperature: float = 0.7) -> List[Dict]:
        """
        Process multiple queries in batch
        
        Args:
            queries (List[str]): List of poker queries
            temperature (float): Temperature setting for responses
            
        Returns:
            List[Dict]: List of responses for each query
        """
        return [self.get_strategy_advice(query, temperature) for query in queries]
